In [29]:
if ebs_volume_ids and ebs_volume_type and not region:
    raise SystemExit("Provide a region for the EBS Volumes!")
elif not ebs_volume_type and region and ebs_volume_ids:
    raise SystemExit("Provide a Volume ID!")
elif not ebs_volume_ids and region and ebs_volume_type:
    raise SystemExit("Provide a Volume Type !")
elif not ebs_volume_ids and not region and ebs_volume_type:
    raise SystemExit("Provide a Volume Type and region!")
elif not ebs_volume_type and not region and ebs_volume_ids:
    raise SystemExit("Provide a Volume Type and region !")

<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective</h3>
<br><strong style="color: #000000;"><em>Change EBS volumes that are not GP3 Type to GP3 Type</em></strong></div>
</center>
<p>&nbsp;</p>
<center>
<h2 id="Secure-Publicly-accessible-Amazon-RDS-Snapshot"><u><strong style="color: #000000;">Change AWS EBS volume to GP3 Type</strong></u></h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p><br>1)<a href="#1"> Get AWS EBS Volume Without GP3 Type</a><br>2)<a href="#2"> Modify EBS Volume to GP3</a></p>

<h3 id="List-all-AWS-Regions">List all AWS Regions</h3>
<p>This action fetches all AWS Regions to execute Step 1👇. This action will only execute if no <code>region</code> is provided.</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following ouput: <code>region</code></p>
</blockquote>

In [4]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Dict, List
import pprint

from beartype import beartype
@beartype
def aws_list_all_regions_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_list_all_regions(handle) -> List:
    """aws_list_all_regions lists all the AWS regions

        :type handle: object
        :param handle: Object returned from Task Validate

        :rtype: Result List of result
    """

    result = handle.aws_cli_command("aws ec2 --region us-west-2 describe-regions --all-regions --query 'Regions[].{Name:RegionName}' --output text")
    if result is None or result.returncode != 0:
        print("Error while executing command : {}".format(result))
        return str()
    result_op = list(result.stdout.split("\n"))
    list_region = [x for x in result_op if x != '']
    return list_region


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not region",
    "condition_result": true
    }''')
task.configure(outputName="region")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_list_all_regions, lego_printer=aws_list_all_regions_printer, hdl=hdl, args=args)

<h3 id="Get-AWS-EBS-Volume-Without-GP3-Type"><a id="2" target="_self" rel="nofollow"></a>Get AWS EBS Volume Without GP3 Type<a class="jp-InternalAnchorLink" href="#Get-AWS-EBS-Volume-Without-GP3-Type" target="_self">&para;</a></h3>
<p>Using unSkript's Get AWS EBS Volume Without GP3 Type action we will fetch all the EBS Volumes that are not of General Purpose Type-3.</p>
<blockquote>
<p>This action takes the following parameters: <code>region(Optional)</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output: <code>all_volumes</code></p>
</blockquote>

In [5]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Optional, Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint


from beartype import beartype
@beartype
def aws_get_ebs_volumes_without_gp3_type_printer(output):
    if output is None:
        return

    pprint.pprint(output)


@beartype
def aws_get_ebs_volumes_without_gp3_type(handle, region: str = "") -> Tuple:
    """aws_get_ebs_volumes_without_gp3_type Returns an array of ebs volumes.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :rtype: Tuple with status result and list of EBS Volume without GP3 type.
    """
    result=[]
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)

    for reg in all_regions:
        try:
            # Filtering the volume by region
            ec2Client = handle.resource('ec2', region_name=reg)
            volumes = ec2Client.volumes.all()

            # collecting the volumes which has zero attachments
            for volume in volumes:
                volume_dict = {}
                if volume.volume_type != "gp3":
                    volume_dict["region"] = reg
                    volume_dict["volume_id"] = volume.id
                    volume_dict["volume_type"] = volume.volume_type
                    result.append(volume_dict)
        except Exception as e:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, [])


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "region",
    "iter_parameter": "region"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not ebs_volume_ids and not ebs_volume_type",
    "condition_result": true
    }''')
task.configure(outputName="all_volumes")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_ebs_volumes_without_gp3_type, lego_printer=aws_get_ebs_volumes_without_gp3_type_printer, hdl=hdl, args=args)

<h3 id="Create-List-of-Volumes-without-GP3-Type">Create List of Volumes without GP3 Type<a class="jp-InternalAnchorLink" href="#Create-List-of-Volumes-without-GP3-Type" target="_self">&para;</a></h3>
<p>This action filters regions that have no ebs volumes without gp3 type .</p>
<blockquote>
<p>This action takes the following parameters: <code>None</code></p>
</blockquote>
<blockquote>
<p>This action captures the following output:&nbsp;<code>all_non_gp3_volumes</code></p>
</blockquote>

In [6]:
all_non_gp3_volumes = []
dummy = []
try:
    for reg,res in all_volumes.items():
        if res[0]==False:
            if len(res[1])!=0:
                dummy = res[1]
                for x in dummy:
                    all_non_gp3_volumes.append(x)
except Exception:
    for vol_id in ebs_volume_ids:
        data_dict = {}
        data_dict["region"] = region[0]
        data_dict["volume_type"] = ebs_volume_type
        data_dict["volume_id"] = vol_id
        all_non_gp3_volumes.append(data_dict)
print(all_non_gp3_volumes)
task.configure(outputName="all_non_gp3_volumes")

<h3 id="Modify-EBS-Volume-to-GP3&para;">Modify EBS Volume to GP3</h3>
<p>Using unSkript's Modify EBS Volume to GP3 action we will modify the EBS volume type to GP3.</p>
<blockquote>
<p>This action takes the following parameters: <code>region</code>, &nbsp;<code>volume_type,&nbsp;volume_id</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import List
import pprint


from beartype import beartype
@beartype
def aws_modify_ebs_volume_to_gp3_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_modify_ebs_volume_to_gp3(handle, region: str, volume_id: str) -> List:
    """aws_modify_ebs_volume_to_gp3 returns an array of modified details for EBS volumes.

        :type region: string
        :param region: Used to filter the volume for specific region.

        :type volume_id: string
        :param volume_id: EBS Volume ID.

        :rtype: List of modified details for EBS volumes
    """
    result = []
    try:
        ec2Client = handle.client('ec2', region_name=region)
        volumes = ec2Client.modify_volume(VolumeId=volume_id, VolumeType='gp3')
        result.append(volumes)
    except Exception as e:
        result.append({"error": e})

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "iter.get(\\"region\\")",
    "volume_id": "iter.get(\\"volume_id\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_non_gp3_volumes",
    "iter_parameter": ["region","volume_id"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_non_gp3_volumes)!=0",
    "condition_result": true
    }''')
task.configure(outputName="modified_volumes")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_modify_ebs_volume_to_gp3, lego_printer=aws_modify_ebs_volume_to_gp3_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self">&para;</a></h3>
<p>In this Runbook, we were able to change the type of those EBS volumes that weren't type GP3 to type GP3 by using unSkript's AWS actions. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>